# 🌍 Karachi AQI Predictor — Exploratory Data Analysis

This notebook performs a full EDA on the AQI feature dataset. It covers:

1. **Data Loading & Overview** — shape, dtypes, missing values
2. **Target Distribution** — AQI class balance
3. **Pollutant Analysis** — distributions, outliers, correlations
4. **Weather Analysis** — temperature, humidity, wind, pressure
5. **Time-Based Trends** — hourly, daily, monthly, seasonal patterns
6. **Feature Correlations** — heatmap + pairplot
7. **Derived Feature Analysis** — pm_ratio, temp_range, aqi_change_rate
8. **Outlier Detection** — IQR-based flagging
9. **Key Insights Summary**

---
> **Data source:** Generated by loaded from Hopsworks Feature Store.

## 0. Setup & Imports

In [ ]:
import os
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
load_dotenv()

# ── Plot style ────────────────────────────────────────────────────────────────
plt.rcParams.update({
    'figure.facecolor': '#0E1117',
    'axes.facecolor':   '#1A1F2E',
    'axes.edgecolor':   '#444',
    'axes.labelcolor':  'white',
    'xtick.color':      'white',
    'ytick.color':      'white',
    'text.color':       'white',
    'grid.color':       '#333',
    'grid.linestyle':   '--',
    'grid.alpha':       0.5,
    'font.size':        11,
})

PALETTE = ['#00E400', '#FFFF00', '#FF7E00', '#FF0000', '#8F3F97']
AQI_LABELS = {1: 'Good', 2: 'Moderate', 3: 'Unhealthy (SG)', 4: 'Unhealthy', 5: 'Very Unhealthy'}

print('✅ Setup complete')

## 1. Data Loading

In [ ]:
def load_data() -> pd.DataFrame:
    """Load data from Hopsworks Feature Store, falling back to local CSVs."""

    # ── Try Hopsworks first ───────────────────────────────────────────────────
    hops_key     = os.getenv('HOPSWORKS_API_KEY')
    hops_project = os.getenv('HOPSWORKS_PROJECT_NAME')
    fg_name      = os.getenv('FEATURE_GROUP_NAME',    'karachi_aqi_features')
    fg_version   = int(os.getenv('FEATURE_GROUP_VERSION', '1'))

    if hops_key and hops_project:
        try:
            import hopsworks
            print('Connecting to Hopsworks...')
            project = hopsworks.login(api_key_value=hops_key, project=hops_project)
            fs  = project.get_feature_store()
            fg  = fs.get_feature_group(name=fg_name, version=fg_version)
            df  = fg.read()
            print(f'✅ Loaded {len(df):,} records from Hopsworks')
            return df
        except Exception as e:
            print(f'⚠️  Hopsworks unavailable ({e}), falling back to local CSV')

    # ── Fall back to local CSVs ───────────────────────────────────────────────
    csv_dir = Path('data/features')
    if not csv_dir.exists():
        raise FileNotFoundError(
            'No data found. Run backfill_pipeline.py first:\n'
            '    python backfill_pipeline.py'
        )
    files = list(csv_dir.glob('*.csv'))
    if not files:
        raise FileNotFoundError(f'No CSV files found in {csv_dir}')

    df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True).drop_duplicates()
    print(f'✅ Loaded {len(df):,} records from {len(files)} local CSV file(s)')
    return df


df_raw = load_data()

# Parse timestamp
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
df = df_raw.copy()

print(f'Shape : {df.shape}')
print(f'Period: {df["timestamp"].min()}  →  {df["timestamp"].max()}')

## 2. Data Overview

In [ ]:
print('── dtypes & non-null counts ─────────────────────────────────────────────')
print(df.info())

In [ ]:
print('── Descriptive statistics ───────────────────────────────────────────────')
df.describe().round(3)

In [ ]:
# Missing-value heatmap
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df  = pd.DataFrame({'missing': missing, 'pct': missing_pct})
missing_df  = missing_df[missing_df['missing'] > 0]

if missing_df.empty:
    print('✅ No missing values found in the dataset!')
else:
    fig, ax = plt.subplots(figsize=(10, 4))
    missing_df['pct'].sort_values().plot(kind='barh', ax=ax, color='#e74c3c')
    ax.set_title('Missing Values (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('% missing')
    ax.grid(axis='x')
    plt.tight_layout()
    plt.show()
    print(missing_df)

## 3. Target Variable — AQI Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('AQI Target Distribution', fontsize=15, fontweight='bold', y=1.02)

# Bar chart of class counts
aqi_counts = df['aqi'].value_counts().sort_index()
aqi_labels = [AQI_LABELS.get(int(k), str(k)) for k in aqi_counts.index]
bars = axes[0].bar(aqi_labels, aqi_counts.values,
                   color=PALETTE[:len(aqi_counts)], edgecolor='white', linewidth=0.5)
for bar, count in zip(bars, aqi_counts.values):
    axes[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5,
                 str(count), ha='center', va='bottom', fontsize=10)
axes[0].set_title('Class Frequency')
axes[0].set_xlabel('AQI Category')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=15)

# Pie chart
axes[1].pie(
    aqi_counts.values,
    labels=aqi_labels,
    colors=PALETTE[:len(aqi_counts)],
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
    textprops={'color': 'white'}
)
axes[1].set_title('Class Share')

plt.tight_layout()
plt.savefig('eda_aqi_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nClass balance:')
print(pd.DataFrame({'count': aqi_counts, 'pct': (aqi_counts / len(df) * 100).round(1)}))

## 4. Pollutant Analysis

In [ ]:
POLLUTANTS = ['pm2_5', 'pm10', 'no2', 'o3', 'so2', 'co']
POLL_LABELS = ['PM₂.₅', 'PM₁₀', 'NO₂', 'O₃', 'SO₂', 'CO']
POLL_COLORS = ['#e74c3c', '#e67e22', '#f1c40f', '#3498db', '#9b59b6', '#1abc9c']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Pollutant Distributions (μg/m³)', fontsize=15, fontweight='bold')

for ax, col, label, color in zip(axes.flat, POLLUTANTS, POLL_LABELS, POLL_COLORS):
    data = df[col].dropna()
    ax.hist(data, bins=30, color=color, edgecolor='white', linewidth=0.3, alpha=0.85)
    ax.axvline(data.mean(),   color='white',  linestyle='--', linewidth=1.5, label=f'Mean {data.mean():.1f}')
    ax.axvline(data.median(), color='yellow', linestyle=':',  linewidth=1.5, label=f'Median {data.median():.1f}')
    ax.set_title(label, fontsize=12)
    ax.set_xlabel('μg/m³')
    ax.set_ylabel('Count')
    ax.legend(fontsize=8)
    ax.grid(True)

plt.tight_layout()
plt.savefig('eda_pollutant_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Boxplot: pollutant levels grouped by AQI category
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Pollutant Levels by AQI Category', fontsize=15, fontweight='bold')

df['aqi_label'] = df['aqi'].map(AQI_LABELS).fillna(df['aqi'].astype(str))
order = [v for v in AQI_LABELS.values() if v in df['aqi_label'].unique()]

for ax, col, label, color in zip(axes.flat, POLLUTANTS, POLL_LABELS, POLL_COLORS):
    sns.boxplot(data=df, x='aqi_label', y=col, order=order,
                palette=PALETTE[:len(order)], ax=ax,
                flierprops={'markerfacecolor': color, 'markersize': 3})
    ax.set_title(label)
    ax.set_xlabel('AQI Category')
    ax.set_ylabel('μg/m³')
    ax.tick_params(axis='x', rotation=20)
    ax.grid(True, axis='y')

plt.tight_layout()
plt.savefig('eda_pollutant_by_aqi.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Weather Feature Analysis

In [ ]:
WEATHER_COLS   = ['temperature', 'humidity', 'wind_speed', 'pressure']
WEATHER_LABELS = ['Temperature (°C)', 'Humidity (%)', 'Wind Speed (m/s)', 'Pressure (hPa)']
WEATHER_COLORS = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6']

fig, axes = plt.subplots(2, 2, figsize=(14, 9))
fig.suptitle('Weather Feature Distributions', fontsize=15, fontweight='bold')

for ax, col, label, color in zip(axes.flat, WEATHER_COLS, WEATHER_LABELS, WEATHER_COLORS):
    data = df[col].dropna()
    ax.hist(data, bins=30, color=color, edgecolor='white', linewidth=0.3, alpha=0.85)
    ax.axvline(data.mean(),   color='white',  linestyle='--', linewidth=1.5, label=f'Mean {data.mean():.1f}')
    ax.axvline(data.median(), color='yellow', linestyle=':',  linewidth=1.5, label=f'Median {data.median():.1f}')
    ax.set_title(label)
    ax.set_xlabel(label)
    ax.set_ylabel('Count')
    ax.legend(fontsize=9)
    ax.grid(True)

plt.tight_layout()
plt.savefig('eda_weather_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter: Temperature vs PM2.5, coloured by AQI
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Weather vs Pollution Relationships', fontsize=14, fontweight='bold')

scatter_pairs = [
    ('temperature', 'pm2_5',    'Temperature (°C)', 'PM₂.₅ (μg/m³)'),
    ('humidity',    'pm2_5',    'Humidity (%)',      'PM₂.₅ (μg/m³)'),
]

for ax, (xcol, ycol, xlabel, ylabel) in zip(axes, scatter_pairs):
    for aqi_val, color in zip(sorted(df['aqi'].unique()), PALETTE):
        mask = df['aqi'] == aqi_val
        ax.scatter(df.loc[mask, xcol], df.loc[mask, ycol],
                   alpha=0.6, s=30, color=color,
                   label=AQI_LABELS.get(int(aqi_val), str(aqi_val)))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(f'{ylabel} vs {xlabel}')
    ax.legend(title='AQI', fontsize=8, title_fontsize=9)
    ax.grid(True)

plt.tight_layout()
plt.savefig('eda_weather_vs_pollution.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Time-Based Trends

In [ ]:
# Hourly AQI pattern
hourly = df.groupby('hour')['aqi'].agg(['mean', 'std']).reset_index()

fig, ax = plt.subplots(figsize=(13, 5))
ax.plot(hourly['hour'], hourly['mean'], color='#00E400', linewidth=2.5, marker='o', markersize=6)
ax.fill_between(hourly['hour'],
                hourly['mean'] - hourly['std'],
                hourly['mean'] + hourly['std'],
                alpha=0.2, color='#00E400', label='±1 std dev')
# Rush hour shading
for rh_start, rh_end in [(7, 9), (17, 19)]:
    ax.axvspan(rh_start, rh_end, alpha=0.12, color='#FF7E00', label='Rush hour' if rh_start == 7 else '')
ax.set_title('Average AQI by Hour of Day', fontsize=14, fontweight='bold')
ax.set_xlabel('Hour')
ax.set_ylabel('Mean AQI')
ax.set_xticks(range(0, 24))
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.savefig('eda_hourly_trend.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Day-of-week and monthly AQI patterns
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('AQI Patterns by Day and Month', fontsize=14, fontweight='bold')

# Day of week
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow = df.groupby('day_of_week')['aqi'].mean().reindex(range(7))
bars = axes[0].bar(dow_labels, dow.values,
                   color=['#FF7E00' if i >= 5 else '#2E75B6' for i in range(7)],
                   edgecolor='white', linewidth=0.5)
axes[0].axhline(dow.mean(), color='yellow', linestyle='--', linewidth=1.5, label=f'Overall mean {dow.mean():.2f}')
axes[0].set_title('Mean AQI by Day of Week')
axes[0].set_ylabel('Mean AQI')
axes[0].legend()
axes[0].grid(True, axis='y')
# Annotate weekend bars
for bar, label in zip(bars[5:], ['Sat', 'Sun']):
    axes[0].text(bar.get_x() + bar.get_width()/2, 0.05, 'Weekend',
                 ha='center', va='bottom', fontsize=8, color='white', rotation=90)

# Monthly
month_labels = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly = df.groupby('month')['aqi'].mean()
axes[1].bar([month_labels[m-1] for m in monthly.index], monthly.values,
            color='#2E75B6', edgecolor='white', linewidth=0.5)
axes[1].axhline(monthly.mean(), color='yellow', linestyle='--', linewidth=1.5,
                label=f'Overall mean {monthly.mean():.2f}')
axes[1].set_title('Mean AQI by Month')
axes[1].set_ylabel('Mean AQI')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()
axes[1].grid(True, axis='y')

plt.tight_layout()
plt.savefig('eda_dow_monthly_trend.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Seasonal AQI boxplot
season_labels = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
df['season_label'] = df['season'].map(season_labels)

fig, ax = plt.subplots(figsize=(10, 5))
season_order = [v for k, v in sorted(season_labels.items()) if v in df['season_label'].unique()]
sns.boxplot(data=df, x='season_label', y='aqi', order=season_order,
            palette=['#3498db', '#2ecc71', '#e74c3c', '#e67e22'], ax=ax)
ax.set_title('AQI Distribution by Season', fontsize=14, fontweight='bold')
ax.set_xlabel('Season')
ax.set_ylabel('AQI')
ax.grid(True, axis='y')
plt.tight_layout()
plt.savefig('eda_seasonal_aqi.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# AQI over time (rolling 7-day average)
df_time = df.set_index('timestamp').sort_index()

fig, ax = plt.subplots(figsize=(14, 5))
ax.scatter(df_time.index, df_time['aqi'], alpha=0.3, s=15, color='#2E75B6', label='Daily AQI')
rolling = df_time['aqi'].rolling('7D').mean()
ax.plot(rolling.index, rolling.values, color='#FF7E00', linewidth=2.5, label='7-day rolling mean')
ax.set_title('AQI Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('AQI')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.savefig('eda_aqi_over_time.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Feature Correlation Analysis

In [ ]:
NUM_COLS = [
    'pm2_5', 'pm10', 'no2', 'o3', 'so2', 'co',
    'temperature', 'humidity', 'wind_speed', 'pressure',
    'pm_ratio', 'temp_range', 'aqi_change_rate', 'aqi'
]
# Keep only columns that exist in the loaded dataframe
NUM_COLS = [c for c in NUM_COLS if c in df.columns]

corr = df[NUM_COLS].corr()

fig, ax = plt.subplots(figsize=(14, 11))
mask = np.triu(np.ones_like(corr, dtype=bool))   # upper triangle mask

sns.heatmap(
    corr, mask=mask, annot=True, fmt='.2f', linewidths=0.5,
    cmap='RdYlGn', center=0, vmin=-1, vmax=1,
    annot_kws={'size': 8}, ax=ax,
    cbar_kws={'shrink': 0.8, 'label': 'Pearson r'}
)
ax.set_title('Feature Correlation Heatmap', fontsize=15, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('eda_correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

# Top correlations with AQI
print('\nTop correlations with AQI target:')
corr_aqi = corr['aqi'].drop('aqi').abs().sort_values(ascending=False)
print(corr_aqi.head(10).round(3).to_string())

In [ ]:
# Pairplot of top 5 features vs AQI
top5 = corr_aqi.head(5).index.tolist()
pp_cols = top5 + ['aqi']

pp = sns.pairplot(
    df[pp_cols].dropna(),
    hue='aqi',
    palette=dict(zip(sorted(df['aqi'].unique()), PALETTE)),
    plot_kws={'alpha': 0.4, 's': 20},
    diag_kind='kde',
    corner=True,
)
pp.figure.suptitle(f'Pairplot: Top 5 Features vs AQI', y=1.01, fontsize=13, fontweight='bold', color='white')
pp.figure.patch.set_facecolor('#0E1117')
for ax in pp.axes.flat:
    if ax is not None:
        ax.set_facecolor('#1A1F2E')
plt.savefig('eda_pairplot.png', dpi=120, bbox_inches='tight')
plt.show()

## 8. Derived Feature Analysis

In [ ]:
derived_cols   = [c for c in ['pm_ratio', 'temp_range', 'aqi_change_rate'] if c in df.columns]
derived_labels = {'pm_ratio': 'PM Ratio (PM2.5 / PM10+1)',
                  'temp_range': 'Temp Range (°C)',
                  'aqi_change_rate': 'AQI Change Rate'}
derived_colors = ['#e74c3c', '#3498db', '#f39c12']

n = len(derived_cols)
fig, axes = plt.subplots(1, n, figsize=(6 * n, 5))
if n == 1:
    axes = [axes]
fig.suptitle('Derived Feature Distributions', fontsize=14, fontweight='bold')

for ax, col, color in zip(axes, derived_cols, derived_colors):
    data = df[col].dropna()
    ax.hist(data, bins=30, color=color, edgecolor='white', linewidth=0.3, alpha=0.85)
    ax.axvline(data.mean(), color='white', linestyle='--', linewidth=1.5, label=f'Mean {data.mean():.2f}')
    ax.set_title(derived_labels.get(col, col))
    ax.set_ylabel('Count')
    ax.legend(fontsize=9)
    ax.grid(True)

plt.tight_layout()
plt.savefig('eda_derived_features.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# AQI change rate over time — shows pollution spikes
if 'aqi_change_rate' in df.columns:
    df_time2 = df.set_index('timestamp').sort_index()
    fig, ax  = plt.subplots(figsize=(14, 4))
    colors_bar = ['#e74c3c' if v > 0 else '#2ecc71' for v in df_time2['aqi_change_rate']]
    ax.bar(df_time2.index, df_time2['aqi_change_rate'], color=colors_bar, width=0.8, alpha=0.8)
    ax.axhline(0, color='white', linewidth=0.8)
    ax.set_title('AQI Change Rate Over Time  (red = worsening, green = improving)',
                 fontsize=13, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('ΔAQI')
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig('eda_aqi_change_rate.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Outlier Detection

In [ ]:
def detect_outliers_iqr(series: pd.Series, factor: float = 1.5):
    """Return boolean mask of IQR-based outliers."""
    Q1, Q3 = series.quantile(0.25), series.quantile(0.75)
    IQR = Q3 - Q1
    return (series < Q1 - factor * IQR) | (series > Q3 + factor * IQR)

CHECK_COLS = [c for c in POLLUTANTS + ['temperature', 'humidity', 'wind_speed'] if c in df.columns]

outlier_summary = {}
for col in CHECK_COLS:
    mask  = detect_outliers_iqr(df[col].dropna())
    count = mask.sum()
    pct   = count / len(df[col].dropna()) * 100
    outlier_summary[col] = {'outliers': count, 'pct': round(pct, 2)}

out_df = pd.DataFrame(outlier_summary).T.sort_values('pct', ascending=False)
print('IQR-based outlier summary:')
print(out_df.to_string())

# Visualise as boxplots
fig, axes = plt.subplots(2, 4, figsize=(18, 9))
fig.suptitle('Outlier Detection — Boxplots (IQR method)', fontsize=14, fontweight='bold')

for ax, col in zip(axes.flat, CHECK_COLS):
    data = df[col].dropna()
    q1, q3 = data.quantile(0.25), data.quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - 1.5 * iqr, q3 + 1.5 * iqr

    ax.boxplot(data, vert=True, patch_artist=True,
               boxprops={'facecolor': '#2E75B6', 'alpha': 0.7},
               medianprops={'color': 'yellow', 'linewidth': 2},
               flierprops={'marker': 'o', 'color': '#e74c3c', 'alpha': 0.5, 'markersize': 4})
    ax.axhline(low,  color='#e74c3c', linestyle=':', linewidth=1.2, label='IQR bounds')
    ax.axhline(high, color='#e74c3c', linestyle=':', linewidth=1.2)
    ax.set_title(col)
    ax.set_ylabel('Value')
    ax.grid(True, axis='y')

# Hide unused axes
for ax in axes.flat[len(CHECK_COLS):]:
    ax.set_visible(False)

plt.tight_layout()
plt.savefig('eda_outliers_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Rush Hour vs Non-Rush Hour Analysis

In [ ]:
if 'is_rush_hour' in df.columns:
    rush_labels = {0: 'Off-Peak', 1: 'Rush Hour'}
    df['rush_label'] = df['is_rush_hour'].map(rush_labels)

    fig, axes = plt.subplots(1, 2, figsize=(13, 5))
    fig.suptitle('Rush Hour vs Off-Peak Air Quality', fontsize=14, fontweight='bold')

    # Mean pollutant comparison
    rush_poll = df.groupby('rush_label')[POLLUTANTS].mean()
    rush_poll.T.plot(kind='bar', ax=axes[0], color=['#2E75B6', '#e74c3c'],
                     edgecolor='white', linewidth=0.5)
    axes[0].set_title('Mean Pollutant Levels')
    axes[0].set_ylabel('μg/m³')
    axes[0].tick_params(axis='x', rotation=20)
    axes[0].legend(title='')
    axes[0].grid(True, axis='y')

    # AQI distribution comparison
    for label, color in zip(['Off-Peak', 'Rush Hour'], ['#2E75B6', '#e74c3c']):
        subset = df.loc[df['rush_label'] == label, 'aqi'].dropna()
        axes[1].hist(subset, bins=15, alpha=0.65, label=label, color=color,
                     edgecolor='white', linewidth=0.3, density=True)
    axes[1].set_title('AQI Distribution')
    axes[1].set_xlabel('AQI')
    axes[1].set_ylabel('Density')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig('eda_rush_hour.png', dpi=150, bbox_inches='tight')
    plt.show()

    print('Mean AQI — Rush Hour vs Off-Peak:')
    print(df.groupby('rush_label')['aqi'].agg(['mean', 'median', 'std']).round(3))

## 11. Weekend vs Weekday Analysis

In [ ]:
if 'is_weekend' in df.columns:
    df['day_type'] = df['is_weekend'].map({0: 'Weekday', 1: 'Weekend'})

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.violinplot(data=df, x='day_type', y='aqi', palette=['#2E75B6', '#e74c3c'],
                   inner='quartile', ax=ax)
    ax.set_title('AQI Distribution — Weekday vs Weekend', fontsize=13, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('AQI')
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig('eda_weekday_weekend.png', dpi=150, bbox_inches='tight')
    plt.show()

    print('Mean AQI by day type:')
    print(df.groupby('day_type')['aqi'].agg(['mean', 'median', 'std']).round(3))

## 12. Key Insights Summary

In [ ]:
print('=' * 65)
print('  KARACHI AQI — EDA KEY INSIGHTS')
print('=' * 65)

# 1. Class balance
dominant = aqi_counts.idxmax()
dominant_pct = (aqi_counts[dominant] / len(df) * 100)
print(f'\n📊 Target distribution:')
print(f'   Dominant class: AQI={dominant} ({AQI_LABELS.get(int(dominant), "?")}) — {dominant_pct:.1f}% of records')
if dominant_pct > 60:
    print(f'   ⚠️  Class imbalance detected — consider oversampling or class weights')

# 2. Strongest predictor
top_feat = corr_aqi.index[0]
print(f'\n🔗 Strongest correlate with AQI: {top_feat} (|r|={corr_aqi.iloc[0]:.3f})')
print(f'   Top 3: {", ".join(corr_aqi.head(3).index.tolist())}')

# 3. Rush hour impact
if 'rush_label' in df.columns:
    rh_means = df.groupby('rush_label')['aqi'].mean()
    if 'Rush Hour' in rh_means and 'Off-Peak' in rh_means:
        diff = rh_means['Rush Hour'] - rh_means['Off-Peak']
        print(f'\n🚗 Rush hour effect: +{diff:.3f} AQI vs off-peak ({"+" if diff > 0 else ""}{diff/rh_means["Off-Peak"]*100:.1f}%)')

# 4. Worst month
monthly_full = df.groupby('month')['aqi'].mean()
worst_month  = monthly_full.idxmax()
best_month   = monthly_full.idxmin()
month_names  = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
                7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
print(f'\n📅 Seasonal trends:')
print(f'   Worst month: {month_names.get(worst_month, worst_month)} (mean AQI {monthly_full[worst_month]:.2f})')
print(f'   Best month:  {month_names.get(best_month, best_month)} (mean AQI {monthly_full[best_month]:.2f})')

# 5. Outliers
high_outlier_cols = out_df[out_df['pct'] > 5].index.tolist()
if high_outlier_cols:
    print(f'\n⚠️  Columns with >5% outliers: {", ".join(high_outlier_cols)}')
    print(f'   Consider robust scaling or clipping before training')
else:
    print(f'\n✅ No columns with >5% outliers — dataset is relatively clean')

# 6. Weekend vs weekday
if 'day_type' in df.columns:
    dt_means = df.groupby('day_type')['aqi'].mean()
    if 'Weekday' in dt_means and 'Weekend' in dt_means:
        diff_we = dt_means['Weekend'] - dt_means['Weekday']
        direction = 'higher' if diff_we > 0 else 'lower'
        print(f'\n📆 Weekend AQI is {abs(diff_we):.3f} {direction} than weekday average')

# 7. AQI change rate
if 'aqi_change_rate' in df.columns:
    volatility = df['aqi_change_rate'].std()
    print(f'\n📈 AQI change rate std dev: {volatility:.3f} — ', end='')
    print('high volatility' if volatility > 0.5 else 'stable air quality trend')

print('\n' + '=' * 65)
print('  EDA complete. Plots saved as eda_*.png in the working directory.')
print('=' * 65)